# Circuit simulations

`get_netlist()` returns a dict with all the instance settings and connections for a Component

There is another repo [gdslib](https://github.com/gdsfactory/gdslib) that stores all gdsfactory Sparameters (from Lumerical 3D FDTD simulations) and compact models (from [SiPANN library](https://sipann.readthedocs.io/en/latest/)) and allows you to do circuit simulations with [Simphony](https://github.com/BYUCamachoLab/simphony)

This notebook is work in progress

In [ ]:
import pp

delta_length = 10
DL = delta_length/2

c = pp.c.mzi(DL=DL) 
pp.qp(c)

n = c.get_netlist()

`gdslib` has also an MZI model. However we can make sure we get the exact cells from the component

In [ ]:
import gdslib as gl
import pp
from simphony.netlist import Subcircuit

m = gl.components.mzi(DL=DL)
gl.plot_circuit(m)

In this first case we are going to use Sparameters from Lumerical simulations stored in gdslib.
Some simulations (such as waveguides with different lengths and bends with 10um bend radius) may be missing so we ignore all the settings and will use the defaults (bends with `radius = 5` and waveguides with `length = 10`

In [ ]:
component = pp.c.mzi()
circuit = Subcircuit(component.name)
n = component.get_netlist()

model_name_tuple = []
for i in n.instances.keys():
    component_type = n.instances[i]['component']
    component_settings = n.instances[i]['settings']
    c = pp.c.component_type2factory[component_type]()
    #c = pp.c.component_type2factory[component_type](**component_settings)
    model = gl.model_from_gdsfactory(c)
    #print(component_settings)
    #model = gl.component_type2factory[component_type](**component_settings)
    model_name_tuple.append((model, i))
    
e = circuit.add(model_name_tuple)

for k, v in n.connections.items():
    c1, p1 = k.split(',')
    c2, p2 = v.split(',')
    circuit.connect(c1, p1, c2, p2)

circuit.elements["mmi1x2_12_0"].pins["W0"] = "input"
circuit.elements["mmi1x2_88_0"].pins["W0"] = "output"

gl.plot_circuit(circuit)

As we can see from the netlist it's hard to capture all possible variations by doing lots of lumerical simulations. That is when a Compact model for the component becomes more powerful. 

In [ ]:
component = pp.c.mzi()
circuit = Subcircuit(component.name)
n = component.get_netlist()

model_name_tuple = []
for i in n.instances.keys():
    component_type = n.instances[i]['component']
    component_settings = n.instances[i]['settings']
    #c = pp.c.component_type2factory[component_type]()
    #c = pp.c.component_type2factory[component_type](**component_settings)
    # model = gl.model_from_gdsfactory(c)
    #print(component_settings)
    model = gl.component_type2factory[component_type](**component_settings)
    model_name_tuple.append((model, i))
    
e = circuit.add(model_name_tuple)

for k, v in n.connections.items():
    c1, p1 = k.split(',')
    c2, p2 = v.split(',')
    circuit.connect(c1, p1, c2, p2)

circuit.elements["mmi1x2_12_0"].pins["W0"] = "input"
circuit.elements["mmi1x2_88_0"].pins["W0"] = "output"

gl.plot_circuit(circuit)